In [ ]:
import csv

file_path = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\2017-11-26.csv"

address_to_id = {}
next_id = 1
mapped_data = []


with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader) 
    
   
    from_address_column_index = 3 
    to_address_column_index = 4   
    value_column_index = 8        
    from_is_contract_column_index = 6 
    to_is_contract_column_index = 7 
    
    for row in csv_reader:
        from_address = row[from_address_column_index]
        to_address = row[to_address_column_index]
        value = row[value_column_index]
        from_is_contract = row[from_is_contract_column_index]
        to_is_contract = row[to_is_contract_column_index]
        
       
        if from_address not in address_to_id:
            address_to_id[from_address] = next_id
            next_id += 1
        
     
        if to_address not in address_to_id:
            address_to_id[to_address] = next_id
            next_id += 1
        
      
        from_address_id = address_to_id[from_address]
        to_address_id = address_to_id[to_address]
        mapped_data.append([from_address_id, to_address_id, value, from_is_contract, to_is_contract])


output_file_path = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\data.txt"

with open(output_file_path, 'w') as file:
    for entry in mapped_data:
      
        file.write('\t'.join(map(str, entry)) + '\n')

print(f"Data with contract information saved to: {output_file_path}")

Data with contract information saved to: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\data.txt


In [ ]:
import os
import networkx as nx

edge_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\data.txt"  


output_file = os.path.join(os.path.dirname(edge_file), 'largest_connected_subgraph_with_data.txt')

G = nx.DiGraph() 


with open(edge_file, 'r') as f:
    for line in f:
       
        parts = line.strip().split()
        from_node, to_node = parts[0], parts[1]
        value = float(parts[2]) 
        from_is_contract = int(parts[3]) 
        to_is_contract = int(parts[4])
        
       
        G.add_edge(from_node, to_node, weight=value, fromIsContract=from_is_contract, toIsContract=to_is_contract)


weakly_connected_components = nx.weakly_connected_components(G)


largest_component = max(weakly_connected_components, key=len)


largest_subgraph = G.subgraph(largest_component)


with open(output_file, 'w') as f:
    for from_node, to_node, edge_data in largest_subgraph.edges(data=True):
       
        value = edge_data['weight']
        from_is_contract = edge_data['fromIsContract']
        to_is_contract = edge_data['toIsContract']
        
       
        f.write(f"{from_node}\t{to_node}\t{value}\t{from_is_contract}\t{to_is_contract}\n")

print(f"Full data for the largest connected subgraph has been saved to: {output_file}")


Full data for the largest connected subgraph has been saved to: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\largest_connected_subgraph_with_data.txt


In [34]:
import pandas as pd

# Step 1: Read the data
file_path = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\largest_connected_subgraph_with_data.txt"  # Replace with your file path
data = pd.read_csv(file_path, sep='\t', header=None, names=['from', 'to', 'value', 'fromiscontract', 'toiscontract'])

# Step 2: Get unique nodes and assign new node numbers sequentially
unique_nodes = pd.unique(data[['from', 'to']].values.ravel())
node_mapping = {node: idx + 1 for idx, node in enumerate(sorted(unique_nodes))}

# Step 3: Replace the 'from' and 'to' columns with their new node numbers
data['from'] = data['from'].map(node_mapping)
data['to'] = data['to'].map(node_mapping)

# Step 4: Save the modified data to a new text file
output_file_path =os.path.join(os.path.dirname(file_path), 'renumbered_graph.txt')  # Replace with your desired output file path
data.to_csv(output_file_path, sep='\t', index=False, header=False)

print("Renumbered graph data has been saved to:", output_file_path)


Renumbered graph data has been saved to: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\renumbered_graph.txt


In [44]:
import pandas as pd

# Step 1: Read the data
file_path = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\renumbered_graph.txt"  # Replace with your file path
data = pd.read_csv(file_path, sep='\t', header=None, names=['from', 'to', 'value', 'fromiscontract', 'toiscontract'])

# Step 2: Extract only the first two columns
first_two_columns = data[['from', 'to']]

# Step 3: Save the extracted columns to a new text file
output_file_path =os.path.join(os.path.dirname(file_path), 'extracted_columns.txt')  # Replace with your desired output file path
first_two_columns.to_csv(output_file_path, sep='\t', index=False, header=False)

print("First two columns have been saved to:", output_file_path)


First two columns have been saved to: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\extracted_columns.txt


In [ ]:
# Paths to input and output files
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\renumbered_graph.txt"
output_contract_status = os.path.join(os.path.dirname(input_file), "node_contract_status_connected.txt")  


contract_info = {}


with open(input_file, 'r') as file:
    for line in file:
        from_node, to_node, value, from_is_contract, to_is_contract = line.strip().split('\t')
        

        if from_node not in contract_info:
            contract_info[from_node] = int(from_is_contract)
        else:
            contract_info[from_node] = max(contract_info[from_node], int(from_is_contract))

    
        if to_node not in contract_info:
            contract_info[to_node] = int(to_is_contract)
        else:
            contract_info[to_node] = max(contract_info[to_node], int(to_is_contract))


with open(output_contract_status, 'w') as file:
    for node, status in sorted(contract_info.items(), key=lambda x: int(x[0])):  
        file.write(f"{status}\n")  

print(f"Node contract status has been saved to {output_contract_status}")


Node contract status has been saved to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\node_contract_status_connected.txt


In [ ]:
# Path
contract_status_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\node_contract_status_connected.txt"

count_0 = 0
count_1 = 0

with open(contract_status_file, 'r') as file:
    for line in file:
        status = int(line.strip())
        
        if status == 0:
            count_0 += 1
        elif status == 1:
            count_1 += 1

print(f"Number of nodes with contract status 0: {count_0}")
print(f"Number of nodes with contract status 1: {count_1}")


Number of nodes with contract status 0: 182418
Number of nodes with contract status 1: 12831


In [ ]:
import os
import networkx as nx

edge_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\renumbered_graph.txt"
input_file = os.path.join(os.path.dirname(edge_file), r"renumbered_graph.txt")
output_dir = os.path.dirname(input_file) 

print("Input file path:", input_file)

if not os.path.exists(input_file):
    raise FileNotFoundError(f"The input file does not exist: {input_file}")

G = nx.DiGraph()

with open(input_file, 'r') as file:
    for line_number, line in enumerate(file, start=1):
        stripped_line = line.strip()
        if not stripped_line:
            continue
        values = stripped_line.split()
        if len(values) < 5:
            print(f"Skipping line {line_number} due to insufficient values: {stripped_line}")
            continue
        try:
            from_node, to_node, value, _, _ = values
            value = float(value)
            G.add_edge(from_node, to_node, weight=value)
        except ValueError as e:
            print(f"Error processing line {line_number}: {stripped_line} - {e}")


Input file path: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\renumbered_graph.txt


In [153]:
# In-degree
in_degree_dict = dict(G.in_degree())
output_in_degree = os.path.join(output_dir, "indegreeconnected.txt")
with open(output_in_degree, 'w') as file:
    for node, in_degree in sorted(in_degree_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{in_degree}\n")
print(f"In-degree written to {output_in_degree}")


In-degree written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\indegreeconnected.txt


In [154]:
# Out-degree
out_degree_dict = dict(G.out_degree())
output_out_degree = os.path.join(output_dir, "outdegreeconnected.txt")
with open(output_out_degree, 'w') as file:
    for node, out_degree in sorted(out_degree_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{out_degree}\n")
print(f"Out-degree written to {output_out_degree}")


Out-degree written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\outdegreeconnected.txt


In [155]:
# Weighted In-degree
weighted_in_degree_dict = dict(G.in_degree(weight='weight'))
output_weighted_in_degree = os.path.join(output_dir, "weightedindegreeconnected.txt")
with open(output_weighted_in_degree, 'w') as file:
    for node, weighted_in_degree in sorted(weighted_in_degree_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{weighted_in_degree}\n")
print(f"Weighted in-degree written to {output_weighted_in_degree}")


Weighted in-degree written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\weightedindegreeconnected.txt


In [156]:
# Weighted Out-degree
weighted_out_degree_dict = dict(G.out_degree(weight='weight'))
output_weighted_out_degree = os.path.join(output_dir, "weightedoutdegreeconnected.txt")
with open(output_weighted_out_degree, 'w') as file:
    for node, weighted_out_degree in sorted(weighted_out_degree_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{weighted_out_degree}\n")
print(f"Weighted out-degree written to {output_weighted_out_degree}")


Weighted out-degree written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\weightedoutdegreeconnected.txt


In [157]:
# Degree Centrality
degree_centrality_dict = nx.degree_centrality(G)
output_degree_centrality = os.path.join(output_dir, "degreecentralityconnected.txt")
with open(output_degree_centrality, 'w') as file:
    for node, degree_centrality in sorted(degree_centrality_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{degree_centrality}\n")
print(f"Degree centrality written to {output_degree_centrality}")


Degree centrality written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\degreecentralityconnected.txt


In [158]:
# Clustering Coefficient
clustering_coefficient_dict = nx.clustering(G)
output_clustering_coefficient = os.path.join(output_dir, "clusteringcoefficientconnected.txt")
with open(output_clustering_coefficient, 'w') as file:
    for node, clustering_coefficient in sorted(clustering_coefficient_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{clustering_coefficient}\n")
print(f"Clustering coefficient written to {output_clustering_coefficient}")


Clustering coefficient written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\clusteringcoefficientconnected.txt


In [159]:
# PageRank
pagerank_dict = nx.pagerank(G)
output_pagerank = os.path.join(output_dir, "pagerankconnected.txt")
with open(output_pagerank, 'w') as file:
    for node, pagerank in sorted(pagerank_dict.items(), key=lambda x: int(x[0])):
        file.write(f"{pagerank}\n")
print(f"PageRank written to {output_pagerank}")


PageRank written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\pagerankconnected.txt


In [ ]:
import numpy as np

input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\extracted_columns.txt" 
output_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\extractedcolumns1.txt"  

edges = np.loadtxt(input_file, dtype=int)

edges_updated = edges - 1

np.savetxt(output_file, edges_updated, fmt='%d')

print(f"Updated edges saved to {output_file}")


Updated edges saved to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\extractedcolumns1.txt


In [ ]:
import networkx as nx
import numpy as np

# Load the graph from a text file
edge_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\7-11\extractedcolumns1.txt"

try:
    edges = np.loadtxt(edge_file, dtype=int)
except Exception as e:
    print(f"Error reading edge file: {e}")
    exit()

G = nx.DiGraph()
G.add_edges_from(edges)

is_strongly_connected = nx.is_strongly_connected(G)

if is_strongly_connected:
    print("The graph is strongly connected.")
else:
    print("The graph is NOT strongly connected.")

    strongly_connected_components = list(nx.strongly_connected_components(G))
    print(f"Number of Strongly Connected Components: {len(strongly_connected_components)}")
    
if not is_strongly_connected:
    largest_scc = max(strongly_connected_components, key=len)
    print(f"\nLargest Strongly Connected Component Size: {len(largest_scc)}")
    


The graph is NOT strongly connected.
Number of Strongly Connected Components: 148912

Largest Strongly Connected Component Size: 21009


for 2-12 :The graph is NOT strongly connected.
Number of Strongly Connected Components: 221048

Largest Strongly Connected Component Size: 34407

for 6-11:the graph is NOT strongly connected.
Number of Strongly Connected Components: 138265

Largest Strongly Connected Component Size: 18543

for 7-11:The graph is NOT strongly connected.
Number of Strongly Connected Components: 148912

Largest Strongly Connected Component Size: 21009

In [ ]:
import pandas as pd

file_path_1 = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\weightedindegreeconnected.txt"
file_path_2 = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\weightedoutdegreeconnected.txt" 

data1 = pd.read_csv(file_path_1, header=None)
data2 = pd.read_csv(file_path_2, header=None)

result = data1[0] - data2[0]

result = result.apply(lambda x: max(x, 0))

output_file_path  = os.path.join(os.path.dirname(file_path_1), 'subtracted_values.txt') 
result.to_csv(output_file_path, index=False, header=False)

print("Subtracted values with non-negative results have been saved to:", output_file_path)


Subtracted values with non-negative results have been saved to: C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\subtracted_values.txt


In [ ]:
import os

input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\2--12\subtracted_values.txt"

with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  

max_value = max(values)

output_file = os.path.join(os.path.dirname(input_file), 'feature5.txt')

with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\2--12\feature5.txt


In [202]:
import os

# Define the input file path
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\clusteringcoefficientconnected.txt"

# Read the values from the file
with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  # Convert each line to a float

# Find the maximum value
max_value = max(values)

# Create the output file path in the same folder as the input file
output_file = os.path.join(os.path.dirname(input_file), 'feature4.txt')

# Divide each value by the maximum and write to the new file
with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\feature4.txt


In [212]:
import os

# Define the input file path
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\degreecentralityconnected.txt"

# Read the values from the file
with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  # Convert each line to a float

# Find the maximum value
max_value = max(values)

# Create the output file path in the same folder as the input file
output_file = os.path.join(os.path.dirname(input_file), 'feature3.txt')

# Divide each value by the maximum and write to the new file
with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\feature3.txt


In [222]:
import os

# Define the input file path
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\pagerankconnected.txt"

# Read the values from the file
with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  # Convert each line to a float

# Find the maximum value
max_value = max(values)

# Create the output file path in the same folder as the input file
output_file = os.path.join(os.path.dirname(input_file), 'feature6.txt')

# Divide each value by the maximum and write to the new file
with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\feature6.txt


In [235]:
import os

# Define the input file path
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\outdegreeconnected.txt"

# Read the values from the file
with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  # Convert each line to a float

# Find the maximum value
max_value = max(values)

# Create the output file path in the same folder as the input file
output_file = os.path.join(os.path.dirname(input_file), 'feature2.txt')

# Divide each value by the maximum and write to the new file
with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\26-11\feature2.txt


In [248]:
import os

# Define the input file path
input_file = r"C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\2--12\indegreeconnected.txt"

# Read the values from the file
with open(input_file, 'r') as infile:
    values = [float(line.strip()) for line in infile]  # Convert each line to a float

# Find the maximum value
max_value = max(values)

# Create the output file path in the same folder as the input file
output_file = os.path.join(os.path.dirname(input_file), 'feature1.txt')

# Divide each value by the maximum and write to the new file
with open(output_file, 'w') as outfile:
    for value in values:
        normalized_value = value / max_value
        outfile.write(f"{normalized_value}\n")

print(f"Normalized values written to {output_file}")


Normalized values written to C:\Users\varun\OneDrive\Desktop\MINIIII-2\sk\2--12\feature1.txt
